In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
from PIL import Image
import numpy as np
import time

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ECE661_Project/CheXpert/train.csv')
df = df[df['Frontal/Lateral'] == 'Frontal']
df = df[df['AP/PA'] == 'AP']
df

In [ ]:
#display image count per pathology
feature_cols = df.columns[5:]
counts = df[feature_cols].eq(1.0).sum()
counts

In [ ]:
target_cols = ['Cardiomegaly', 'Pleural Effusion', 'Pneumothorax']
mask = df[target_cols].eq(1.0).any(axis=1)

df_3class = df.loc[mask].reset_index(drop=True)[(list(df.columns[:5]) + target_cols)]
df_3class.fillna(0, inplace=True)
print(f"Kept {len(df_3class)} of {len(df)} rows.")
df_3class

In [ ]:
df_3class.to_csv('train_label.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ECE661_Project/train_label.csv')
df

In [ ]:
#saving 60,000 images per batch due to RAM limit
paths = df["Path"]
start_time = time.time()
print(f'number of images to save: {len(paths)}')
image_arrays = []
id_array = []
ct = 60000
for p in paths[60000:]:
    p = '/kaggle/input/chexpert/train' + p.split('train')[-1]
    try:
        with Image.open(p) as img:
          img = img.convert("RGB")
          arr = np.array(img)
    except Exception as e:
        print(f"Warning: skipping {p!r} due to error: {e}")
        continue

    image_arrays.append(arr)
    id_array.append(p.split('train/')[-1].split('.')[0])
    ct += 1
    if ct % 20000 == 0:
      image_arrays = np.array(image_arrays)
      id_array = np.array(id_array)
      np.savez(f"image_train{ct}_1.npz", image=image_arrays, id = id_array)
      print(f"Saved first {ct} images, image size: {image_arrays.shape}")
      image_arrays = []
      id_array = []

image_arrays = np.array(image_arrays)
id_array = np.array(id_array)
np.savez(f"image_train{ct}_1.npz", image=image_arrays, id = id_array)

In [ ]:
#load image for inspection
np.load('/content/drive/MyDrive/ECE661_Project/image_train15.npz')['id'][3]

In [ ]:
dataset1 = np.load('/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/image_train20000.npz')
dataset2= np.load('/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/image_train40000.npz')
dataset3 = np.load('/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/image_train49815.npz')
dataset4 = np.load('/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/image_train80000.npz')
dataset5 = np.load('/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/image_train83937.npz')
ids = [dataset['id'] for dataset in [dataset1, dataset2, dataset3, dataset4, dataset5]]
ids = np.concatenate(ids)

In [ ]:
# dot over the last axis to apply weights → result is (N, H, W)
rgb_float = images.astype(np.float32) / 255.0
weights = np.array([0.2989, 0.5870, 0.1140], dtype=rgb_float.dtype) #convert to grayscale using Luma Coefficients
gray_float = np.tensordot(rgb_float, weights, axes=([-1], [0]))
grayscale_uint8 = (gray_float * 255).astype(np.uint8)

np.save("/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/train_image_greyscale.npy", grayscale_uint8)
loaded = np.load("/content/drive/MyDrive/ECE661_Project/CheXpert-v1.0-small/train_image_greyscale.npy")